In [ ]:
dbutils.fs.ls("/FileStore/tables")

Out[85]: [FileInfo(path='dbfs:/FileStore/tables/authentication_credentials.csv', name='authentication_credentials.csv', size=202, modificationTime=1687110999000),
 FileInfo(path='dbfs:/FileStore/tables/geo_dirty.csv/', name='geo_dirty.csv/', size=0, modificationTime=1701871080155),
 FileInfo(path='dbfs:/FileStore/tables/pin_dirty.csv/', name='pin_dirty.csv/', size=0, modificationTime=1701871080155),
 FileInfo(path='dbfs:/FileStore/tables/single_json_file.json', name='single_json_file.json', size=293, modificationTime=1701857903000),
 FileInfo(path='dbfs:/FileStore/tables/user_dirty.csv/', name='user_dirty.csv/', size=0, modificationTime=1701871080155)]

In [ ]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib
import numpy as np 

In [ ]:
# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

In [ ]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [ ]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0e90e0175553-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/mount1"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

In [ ]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location = "/mnt/mount1/topics/0e90e0175553.geo/partition=0/*.json" 
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
geo_df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
# Display Spark dataframe to check its content
display(geo_df)

country,ind,latitude,longitude,timestamp
Antarctica (the territory South of 60 deg S),2418,-88.4642,-171.061,2022-05-27 11:30:59
Antarctica (the territory South of 60 deg S),2418,-88.4642,-171.061,2022-05-27 11:30:59
Antarctica (the territory South of 60 deg S),5162,-71.6607,-149.206,2019-09-27 19:06:43
Saint Vincent and the Grenadines,2301,13.4683,51.7244,2020-11-14 00:25:28
Saint Vincent and the Grenadines,2301,13.4683,51.7244,2020-11-14 00:25:28
French Southern Territories,6014,-26.6026,155.206,2019-04-30 12:33:13
French Southern Territories,6014,-26.6026,155.206,2019-04-30 12:33:13
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01 02:26:50
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01 02:26:50
Central African Republic,2074,-52.3213,-50.11,2019-11-03 05:41:59


In [ ]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location = "/mnt/mount1/topics/0e90e0175553.pin/partition=0/*.json" 
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
pin_df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
# Display Spark dataframe to check its content
display(pin_df)

category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
christmas,"Features: Material:Lint Size:48ｘ18cm Quantity:1 pc Shape:Santa Claus, snowman. Elk Occasion:Christmas Description: 1. Fashion design, high quality 2. Santa Claus, snowman. Elk C…",1,5k,https://i.pinimg.com/originals/b5/7f/21/b57f219fa89c1165b57525b8eae711da.jpg,1706,image,Wear24-7,Local save in /data/christmas,"Merry Christmas To You,Christmas Toys,Great Christmas Gifts,Christmas Snowman,Christmas Ornaments,Holiday,Christmas Party Decorations,Christmas Themes,Decoration Party",Standing Figurine Toys Xmas Santa Claus Snowman Reindeer Figure Plush Dolls Christmas Decorations Ornaments Home Indoor Table Ornaments Christmas Party Tree Hanging Decor Toys Gifts for Kids Friends…,b5c8a1b5-9e90-4522-9bec-2477b698d5b7
christmas,"Features: Material:Lint Size:48ｘ18cm Quantity:1 pc Shape:Santa Claus, snowman. Elk Occasion:Christmas Description: 1. Fashion design, high quality 2. Santa Claus, snowman. Elk C…",1,5k,https://i.pinimg.com/originals/b5/7f/21/b57f219fa89c1165b57525b8eae711da.jpg,1706,image,Wear24-7,Local save in /data/christmas,"Merry Christmas To You,Christmas Toys,Great Christmas Gifts,Christmas Snowman,Christmas Ornaments,Holiday,Christmas Party Decorations,Christmas Themes,Decoration Party",Standing Figurine Toys Xmas Santa Claus Snowman Reindeer Figure Plush Dolls Christmas Decorations Ornaments Home Indoor Table Ornaments Christmas Party Tree Hanging Decor Toys Gifts for Kids Friends…,b5c8a1b5-9e90-4522-9bec-2477b698d5b7
christmas,"Christmas decorating ideas for porches. Beautiful holiday decor ideas for front porches both small and large. Outdoor decorations like sleds, lanterns, Christmas trees, wreaths,…",1,46k,https://i.pinimg.com/videos/thumbnails/originals/40/83/f5/4083f5b4971bf235f89a4784ab87271e.0000001.jpg,2482,video,"Life on Summerhill | Home, Holiday Decor & DIY Website",Local save in /data/christmas,"Diy Christmas Decorations For Home,Farmhouse Christmas Decor,Christmas Home,Christmas Holidays,Christmas Front Porches,How To Decorate For Christmas,Christmas Porch Ideas,Christmas Decorating Ideas,Large Outdoor Christmas Decorations",FORNT PORCH CHRISTMAS DECORATING IDEAS,08604f20-fa17-4b9a-9949-781717eca6cd
christmas,"Christmas decorating ideas for porches. Beautiful holiday decor ideas for front porches both small and large. Outdoor decorations like sleds, lanterns, Christmas trees, wreaths,…",1,46k,https://i.pinimg.com/videos/thumbnails/originals/40/83/f5/4083f5b4971bf235f89a4784ab87271e.0000001.jpg,2482,video,"Life on Summerhill | Home, Holiday Decor & DIY Website",Local save in /data/christmas,"Diy Christmas Decorations For Home,Farmhouse Christmas Decor,Christmas Home,Christmas Holidays,Christmas Front Porches,How To Decorate For Christmas,Christmas Porch Ideas,Christmas Decorating Ideas,Large Outdoor Christmas Decorations",FORNT PORCH CHRISTMAS DECORATING IDEAS,08604f20-fa17-4b9a-9949-781717eca6cd
event-planning,"Updated: January 25, 2017 You’ve organized some events for your family, friends or community and you have gained a budding reputation for knowing how put events together. You’ve…",1,4k,https://i.pinimg.com/originals/c3/2b/c6/c32bc6ad263857cb0eea19f9cd12beb9.jpg,4357,image,EventPlanning.com | Learn How To Become An Event Planner,Local save in /data/event-planning,"Event Planning Quotes,Event Planning Checklist,Event Planning Business,Business Events,Business Ideas,Business Names,Business Opportunities,Corporate Events,Wedding Event Planner",First Steps in Launching Your Own Event Business - Learn About Event Planning,ccf116e9-9096-4943-a344-1960ce216445
christmas,15 unique Christmas porch ideas that will leave you feeling inspired and help you tackle decorating your own entryway for the holidays! It’s almost time to start decorating for…,1,19k,https://i.pinimg.com/originals/ff/f8/3b/fff83b02aeb29e2e9341a56fc5e63345.png,1967,image,Ashley - Modern Glam,Local sa

In [ ]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location = "/mnt/mount1/topics/0e90e0175553.user/partition=0/*.json" 
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
user_df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
# Display Spark dataframe to check its content
display(user_df)

age,date_joined,first_name,ind,last_name
27,2016-03-08 13:38:37,Christopher,2015,Bradshaw
59,2017-05-12 21:22:17,Alexander,10673,Cervantes
27,2016-03-08 13:38:37,Christopher,2015,Bradshaw
59,2017-05-12 21:22:17,Alexander,10673,Cervantes
39,2016-06-29 20:43:59,Christina,6398,Davenport
20,2015-10-23 04:13:23,Alexandria,3599,Alvarado
39,2016-06-29 20:43:59,Christina,6398,Davenport
20,2015-10-23 04:13:23,Alexandria,3599,Alvarado
20,2015-10-23 04:13:23,Alexandria,4256,Alvarado
20,2015-12-01 15:08:31,Christopher,5076,Butler


Once we have all of the data loaded into tables, we can begin cleaning each table. Each table is displayed.

In [ ]:
df_pin = pin_df.select('*')

df_pin = df_pin.withColumnRenamed('index', 'ind') 
df_pin= df_pin.withColumn( 'save_location', regexp_replace('save_location', 'Local save in ', ''))
df_pin= df_pin.withColumn('follower_count', regexp_replace('follower_count', 'k', '000'))
df_pin= df_pin.withColumn('follower_count', regexp_replace('follower_count', 'M', '000000'))
df_pin= df_pin.withColumn('follower_count', col('follower_count').cast("int"))
df_pin= df_pin.withColumn('ind', col('ind').cast("int"))
df_pin = df_pin.select([when(col(c)=="No description available Story format",None).otherwise(col(c)).alias(c) for c in df_pin.columns])
df_pin = df_pin.select([when(col(c)=="No description available",None).otherwise(col(c)).alias(c) for c in df_pin.columns])
df_pin = df_pin.select([when(col(c)=="Untitled",None).otherwise(col(c)).alias(c) for c in df_pin.columns])
df_pin = df_pin.select([when(col(c)=="No Title Data Available",None).otherwise(col(c)).alias(c) for c in df_pin.columns])
df_pin = df_pin.select([when(col(c)=="User Info Error",None).otherwise(col(c)).alias(c) for c in df_pin.columns])
df_pin = df_pin.select([when(col(c)=="N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",None).otherwise(col(c)).alias(c) for c in df_pin.columns])
df_pin = df_pin.select([when(col(c)=="Image src error.",None).otherwise(col(c)).alias(c) for c in df_pin.columns])
df_pin = df_pin.select('ind', 'unique_id', 'title','description', 'follower_count', 'poster_name', 'tag_list', 'is_image_or_video', 'image_src', 'save_location',  'category')

display(df_pin)

ind,unique_id,title,description,follower_count,poster_name,tag_list,is_image_or_video,image_src,save_location,category
1706,b5c8a1b5-9e90-4522-9bec-2477b698d5b7,Standing Figurine Toys Xmas Santa Claus Snowman Reindeer Figure Plush Dolls Christmas Decorations Ornaments Home Indoor Table Ornaments Christmas Party Tree Hanging Decor Toys Gifts for Kids Friends…,"Features: Material:Lint Size:48ｘ18cm Quantity:1 pc Shape:Santa Claus, snowman. Elk Occasion:Christmas Description: 1. Fashion design, high quality 2. Santa Claus, snowman. Elk C…",5000,Wear24-7,"Merry Christmas To You,Christmas Toys,Great Christmas Gifts,Christmas Snowman,Christmas Ornaments,Holiday,Christmas Party Decorations,Christmas Themes,Decoration Party",image,https://i.pinimg.com/originals/b5/7f/21/b57f219fa89c1165b57525b8eae711da.jpg,/data/christmas,christmas
1706,b5c8a1b5-9e90-4522-9bec-2477b698d5b7,Standing Figurine Toys Xmas Santa Claus Snowman Reindeer Figure Plush Dolls Christmas Decorations Ornaments Home Indoor Table Ornaments Christmas Party Tree Hanging Decor Toys Gifts for Kids Friends…,"Features: Material:Lint Size:48ｘ18cm Quantity:1 pc Shape:Santa Claus, snowman. Elk Occasion:Christmas Description: 1. Fashion design, high quality 2. Santa Claus, snowman. Elk C…",5000,Wear24-7,"Merry Christmas To You,Christmas Toys,Great Christmas Gifts,Christmas Snowman,Christmas Ornaments,Holiday,Christmas Party Decorations,Christmas Themes,Decoration Party",image,https://i.pinimg.com/originals/b5/7f/21/b57f219fa89c1165b57525b8eae711da.jpg,/data/christmas,christmas
2482,08604f20-fa17-4b9a-9949-781717eca6cd,FORNT PORCH CHRISTMAS DECORATING IDEAS,"Christmas decorating ideas for porches. Beautiful holiday decor ideas for front porches both small and large. Outdoor decorations like sleds, lanterns, Christmas trees, wreaths,…",46000,"Life on Summerhill | Home, Holiday Decor & DIY Website","Diy Christmas Decorations For Home,Farmhouse Christmas Decor,Christmas Home,Christmas Holidays,Christmas Front Porches,How To Decorate For Christmas,Christmas Porch Ideas,Christmas Decorating Ideas,Large Outdoor Christmas Decorations",video,https://i.pinimg.com/videos/thumbnails/originals/40/83/f5/4083f5b4971bf235f89a4784ab87271e.0000001.jpg,/data/christmas,christmas
2482,08604f20-fa17-4b9a-9949-781717eca6cd,FORNT PORCH CHRISTMAS DECORATING IDEAS,"Christmas decorating ideas for porches. Beautiful holiday decor ideas for front porches both small and large. Outdoor decorations like sleds, lanterns, Christmas trees, wreaths,…",46000,"Life on Summerhill | Home, Holiday Decor & DIY Website","Diy Christmas Decorations For Home,Farmhouse Christmas Decor,Christmas Home,Christmas Holidays,Christmas Front Porches,How To Decorate For Christmas,Christmas Porch Ideas,Christmas Decorating Ideas,Large Outdoor Christmas Decorations",video,https://i.pinimg.com/videos/thumbnails/originals/40/83/f5/4083f5b4971bf235f89a4784ab87271e.0000001.jpg,/data/christmas,christmas
4357,ccf116e9-9096-4943-a344-1960ce216445,First Steps in Launching Your Own Event Business - Learn About Event Planning,"Updated: January 25, 2017 You’ve organized some events for your family, friends or community and you have gained a budding reputation for knowing how put events together. You’ve…",4000,EventPlanning.com | Learn How To Become An Event Planner,"Event Planning Quotes,Event Planning Checklist,Event Planning Business,Business Events,Business Ideas,Business Names,Business Opportunities,Corporate Events,Wedding Event Planner",image,https://i.pinimg.com/originals/c3/2b/c6/c32bc6ad263857cb0eea19f9cd12beb9.jpg,/data/event-planning,event-planning
1967,0b9d5b95-51a6-465e-ae4a-2cb68ceada29,15 Fun & Festive Christmas Porch Ideas,15 unique Christmas porch ideas that will leave you feeling inspired and help you tackle decorating your own entryway for the holidays! It’s almost time to start decorating for…,19000,Ashley - Modern Glam,"Exterior Christmas Lights,Front Door Christmas Decorations,Christmas Lights Outside,Christmas House Lights,

In [ ]:
df_geo = geo_df.select('*')
 
# df_geo= df_geo.withColumn( 'coordinates', zip_with("latitude", "longitude"))
df_geo= df_geo.withColumn('coordinates', struct(df_geo.latitude, df_geo.longitude))
df_geo = df_geo.drop('latitude') 
df_geo = df_geo.drop('longitude') 
df_geo= df_geo.withColumn('timestamp', col('timestamp').cast("timestamp"))
df_geo = df_geo.select('ind', 'country', 'coordinates','timestamp')
display(df_geo)

ind,country,coordinates,timestamp
2418,Antarctica (the territory South of 60 deg S),"List(-88.4642, -171.061)",2022-05-27T11:30:59.000+0000
2418,Antarctica (the territory South of 60 deg S),"List(-88.4642, -171.061)",2022-05-27T11:30:59.000+0000
5162,Antarctica (the territory South of 60 deg S),"List(-71.6607, -149.206)",2019-09-27T19:06:43.000+0000
2301,Saint Vincent and the Grenadines,"List(13.4683, 51.7244)",2020-11-14T00:25:28.000+0000
2301,Saint Vincent and the Grenadines,"List(13.4683, 51.7244)",2020-11-14T00:25:28.000+0000
6014,French Southern Territories,"List(-26.6026, 155.206)",2019-04-30T12:33:13.000+0000
6014,French Southern Territories,"List(-26.6026, 155.206)",2019-04-30T12:33:13.000+0000
10794,Cocos (Keeling) Islands,"List(-89.5236, -154.567)",2022-01-01T02:26:50.000+0000
10794,Cocos (Keeling) Islands,"List(-89.5236, -154.567)",2022-01-01T02:26:50.000+0000
2074,Central African Republic,"List(-52.3213, -50.11)",2019-11-03T05:41:59.000+0000


In [ ]:
df_user = user_df.select('*')
 
# df_user= df_user.withColumn( 'coordinates', zip_with("latitude", "longitude"))
df_user= df_user.withColumn( 'user_name', concat( 'first_name', lit(' '),'last_name'))
df_user = df_user.drop('first_name') 
df_user = df_user.drop('last_name') 
df_user= df_user.withColumn('date_joined', col('date_joined').cast("timestamp"))
df_user= df_user.withColumn('age', col('age').cast("int"))
df_user = df_user.select('ind', 'user_name', 'age','date_joined')
display(df_user)

ind,user_name,age,date_joined
2015,Christopher Bradshaw,27,2016-03-08T13:38:37.000+0000
10673,Alexander Cervantes,59,2017-05-12T21:22:17.000+0000
2015,Christopher Bradshaw,27,2016-03-08T13:38:37.000+0000
10673,Alexander Cervantes,59,2017-05-12T21:22:17.000+0000
6398,Christina Davenport,39,2016-06-29T20:43:59.000+0000
3599,Alexandria Alvarado,20,2015-10-23T04:13:23.000+0000
6398,Christina Davenport,39,2016-06-29T20:43:59.000+0000
3599,Alexandria Alvarado,20,2015-10-23T04:13:23.000+0000
4256,Alexandria Alvarado,20,2015-10-23T04:13:23.000+0000
5076,Christopher Butler,20,2015-12-01T15:08:31.000+0000


From this point, all of the tables have been cleaned and are ready to query.

In [ ]:
df_geo.createOrReplaceTempView("df_geot")
df_pin.createOrReplaceTempView("df_pint")
df_user.createOrReplaceTempView("df_usert")

query1 = """
with data as 
(SELECT g.country as country, p.category as category, COUNT(category) as ttt, RANK() OVER (PARTITION BY country ORDER BY COUNT(category) DESC) rank
FROM df_geot as g
JOIN df_pint as p
ON g.ind == p.ind 
GROUP BY country, category
ORDER BY country, ttt DESC )

SELECT country, category
FROM data
WHERE rank = 1  
ORDER BY country  
""" 

result = spark.sql(query1)
result.show()
 

+--------------------+--------------+
 country| category|
+--------------------+--------------+
 Afghanistan| education|
 Albania| mens-fashion|
 Algeria| quotes|
 American Samoa| beauty|
 Andorra| tattoos|
 Andorra| quotes|
 Andorra| art|
 Andorra| beauty|
 Angola|diy-and-crafts|
 Anguilla|diy-and-crafts|
 Anguilla| beauty|
 Anguilla| home-decor|
Antarctica (the t...| christmas|
 Antigua and Barbuda| christmas|
 Argentina|diy-and-crafts|
 Argentina| beauty|
 Argentina| tattoos|
 Armenia|diy-and-crafts|
 Aruba| mens-fashion|
 Australia| mens-fashion|
+--------------------+--------------+
only showing top 20 rows

In [ ]:
query2 = """
SELECT YEAR(g.timestamp) as year, p.category, COUNT(p.category) as category_count
FROM df_geot as g
JOIN df_pint as p
ON g.ind == p.ind 
WHERE 2017 < YEAR(g.timestamp)
GROUP BY p.category, year
ORDER BY year, p.category DESC
""" 

result = spark.sql(query2)
result.show()

+----+--------------+--------------+
year| category|category_count|
+----+--------------+--------------+
2018| vehicles| 9|
2018| travel| 16|
2018| tattoos| 4|
2018| quotes| 18|
2018| mens-fashion| 8|
2018| finance| 4|
2018| education| 15|
2018|diy-and-crafts| 14|
2018| christmas| 12|
2018| beauty| 16|
2018| art| 21|
2019| vehicles| 8|
2019| travel| 17|
2019| quotes| 4|
2019| mens-fashion| 9|
2019| home-decor| 10|
2019| finance| 9|
2019|event-planning| 13|
2019| education| 17|
2019|diy-and-crafts| 25|
+----+--------------+--------------+
only showing top 20 rows

In [ ]:
query3 = """
with data as (
    SELECT g.country as country, g.ind as iind, p.poster_name as pn, p.follower_count as ttt, RANK() OVER (PARTITION BY country ORDER BY p.follower_count DESC) rank  
    FROM df_geot as g
    JOIN df_pint as p
    ON g.ind == p.ind 
    GROUP BY country, iind, pn, ttt
    ORDER BY country, ttt DESC
) 
SELECT country, pn, ttt as follower_count
FROM data 
WHERE rank = 1
GROUP BY country, pn, ttt  
ORDER BY follower_count DESC
""" 

result = spark.sql(query3)
result.show() 

+--------------------+--------------------+--------------+
 country| pn|follower_count|
+--------------------+--------------------+--------------+
 American Samoa| Mamas Uncut| 8000000|
 Azerbaijan| Style Me Pretty| 6000000|
 Albania| The Minds Journal| 5000000|
 Afghanistan| 9GAG| 3000000|
 Christmas Island| Instructables| 3000000|
 Bangladesh|Architectural Digest| 3000000|
 Antigua and Barbuda|Country Living Ma...| 1000000|
 Australia| Cultura Colectiva| 1000000|
 Andorra|Teachers Pay Teac...| 1000000|
 Algeria| YourTango| 942000|
 Lesotho| The Guardian| 908000|
 Armenia|Michelle {CraftyM...| 892000|
 Aruba| GQ Magazine| 874000|
Central African R...| PureWow| 868000|
 Argentina| Next Luxury| 800000|
 Pitcairn Islands| Marla Meridith| 671000|
 Bulgaria| A Piece of Rainbow| 603000|
 Angola| CraftGossip.com| 502000|
Antarctica (the t...| HikenDip| 500000|
 Barbados|The Creativity Ex...| 410000|
+--------------------+--------------------+--------------+
only showing top 20 rows

+--------------+-----------+--------------+
 country| pn|follower_count|
+--------------+-----------+--------------+
American Samoa|Mamas Uncut| 8000000|
+--------------+-----------+--------------+
only showing top 1 row

In [ ]:
query4 = """
SELECT country, pn as postername, follower_count
FROM (SELECT country, pn, ttt as follower_count, ROW_NUMBER() OVER (ORDER BY ttt DESC) as ranked
    FROM (SELECT g.country as country, g.ind as iind, p.poster_name as pn, p.follower_count as ttt, RANK() OVER (PARTITION BY country ORDER BY p.follower_count DESC) rank  
        FROM df_geot as g
        JOIN df_pint as p
        ON g.ind == p.ind 
        GROUP BY country, iind, pn, ttt
        ORDER BY country, ttt DESC
    ) 
    WHERE rank = 1
    GROUP BY country, pn, ttt  
    ORDER BY follower_count DESC)  
WHERE ranked = 1
""" 

result = spark.sql(query4)
result.show() 

+--------------+-----------+--------------+
 country| postername|follower_count|
+--------------+-----------+--------------+
American Samoa|Mamas Uncut| 8000000|
+--------------+-----------+--------------+

In [ ]:
query5 = """ 
with data as (
SELECT CASE WHEN u.age <= 25 THEN '18-24' 
            WHEN u.age <= 35 THEN '25-35'  
            WHEN u.age <= 50 THEN '35-50' 
            WHEN u.age > 50 THEN '50+'
        END as age, p.category as category   	
FROM df_usert as u   
JOIN df_pint as p
ON u.ind == p.ind  
)

SELECT age, category, ttt as category_count
  FROM(
    SELECT age, category, COUNT(category) as ttt, RANK() OVER (PARTITION BY age ORDER BY COUNT(data.category) DESC) rank 	
    FROM data    
    GROUP BY age, category 
    ORDER BY ttt DESC 
) 
WHERE rank = 1
""" 

result = spark.sql(query5)
result.show()

+-----+--------------+--------------+
 age| category|category_count|
+-----+--------------+--------------+
18-24| christmas| 43|
25-35| christmas| 20|
25-35| art| 20|
35-50|diy-and-crafts| 18|
 50+| beauty| 12|
+-----+--------------+--------------+

In [ ]:
query6 = """
SELECT age, 
    percentile_approx(follower_count, 0.5) as median_follower_count  
FROM  (SELECT CASE WHEN u.age <= 25 THEN '18-24' 
            WHEN u.age <= 35 THEN '25-35'  
            WHEN u.age <= 50 THEN '35-50' 
            WHEN u.age > 50 THEN '50+'
        END as age, p.follower_count as follower_count   	
FROM df_usert as u   
JOIN df_pint as p
ON u.ind == p.ind)    
GROUP BY age 
ORDER BY median_follower_count DESC
""" 

result = spark.sql(query6) 
result.show(result.count())

+-----+---------------------+
 age|median_follower_count|
+-----+---------------------+
18-24| 89000|
25-35| 28000|
35-50| 6000|
 50+| 5000|
+-----+---------------------+

In [ ]:
query7 = """
SELECT year, 
    COUNT(year) as number_users_joined
FROM (
    SELECT YEAR(u.date_joined) as year
    FROM df_usert as u 
    WHERE YEAR(u.date_joined) > 2014 AND YEAR(u.date_joined) < 2021
)
GROUP BY year
ORDER BY year DESC
""" 

result = spark.sql(query7) 
result.show(result.count())

+----+-------------------+
year|number_users_joined|
+----+-------------------+
2017| 50|
2016| 146|
2015| 123|
+----+-------------------+

In [ ]:
query8 = """
SELECT year as post_year, 
    percentile_approx(follower_count, 0.5) as median_follower_count
FROM (
    SELECT YEAR(u.date_joined) as year, p.follower_count
    FROM df_usert as u
    JOIN df_pint as p
    ON u.ind == p.ind 
    WHERE YEAR(u.date_joined) > 2014 AND YEAR(u.date_joined) < 2021
)
GROUP BY post_year
ORDER BY post_year DESC
""" 

result = spark.sql(query8) 
result.show(result.count())

+---------+---------------------+
post_year|median_follower_count|
+---------+---------------------+
 2017| 6000|
 2016| 25000|
 2015| 85000|
+---------+---------------------+

In [ ]:
query9 = """
SELECT age, year, 
    percentile_approx(follower_count, 0.5) as median_follower_count
FROM (
    SELECT YEAR(u.date_joined) as year, p.follower_count, CASE WHEN u.age <= 25 THEN '18-24' 
                                                            WHEN u.age <= 35 THEN '25-35'  
                                                            WHEN u.age <= 50 THEN '35-50' 
                                                            WHEN u.age > 50 THEN '50+'
                                                        END as age
    FROM df_usert as u
    JOIN df_pint as p
    ON u.ind == p.ind 
    WHERE YEAR(u.date_joined) > 2014 AND YEAR(u.date_joined) < 2021
)
GROUP BY age, year
ORDER BY year DESC
""" 

result = spark.sql(query9) 
result.show(result.count())

+-----+----+---------------------+
 age|year|median_follower_count|
+-----+----+---------------------+
18-24|2017| 63000|
25-35|2017| 8000|
35-50|2017| 2000|
 50+|2017| 9000|
35-50|2016| 9000|
25-35|2016| 27000|
 50+|2016| 1000|
18-24|2016| 46000|
18-24|2015| 130000|
35-50|2015| 13000|
 50+|2015| 196|
25-35|2015| 42000|
+-----+----+---------------------+